# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citi_df = pd.read_csv('city_data.csv')
citi_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,esperance,-33.87,121.90,71.60,28,100,13.87,AU,1604169009
1,praia da vitoria,38.73,-27.07,73.40,83,40,12.37,PT,1604169388
2,vestmannaeyjar,63.44,-20.27,44.60,70,20,12.75,IS,1604169218
3,hobart,-42.88,147.33,55.40,66,90,6.93,AU,1604168904
4,valleyview,55.07,-117.29,30.20,74,100,9.17,CA,1604169388
...,...,...,...,...,...,...,...,...,...
603,grenville,12.12,-61.62,78.80,94,75,13.87,GD,1604169474
604,bloomington,44.84,-93.30,55.99,54,75,17.22,US,1604169474
605,midyat,37.42,41.34,69.80,22,75,2.24,TR,1604169474
606,bograd,54.23,90.83,33.80,80,0,4.47,RU,1604169474


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = citi_df[['Lat','Lng']]
humidity = citi_df['Humidity'].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations=locations,weights=humidity,dissipating=False, max_intensity=5,
                                 point_radius=1.2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df = citi_df[['City','Max Temp','Humidity','Wind Speed','Cloudiness']]
new_df.dropna(how='any')

,City,Max Temp,Humidity,Wind Speed,Cloudiness
0,esperance,71.60,28,13.87,100
1,praia da vitoria,73.40,83,12.37,40
2,vestmannaeyjar,44.60,70,12.75,20
3,hobart,55.40,66,6.93,90
4,valleyview,30.20,74,9.17,100
...,...,...,...,...,...
603,grenville,78.80,94,13.87,75
604,bloomington,55.99,54,17.22,75
605,midyat,69.80,22,2.24,75
606,bograd,33.80,80,4.47,0


In [6]:
max_temp = (new_df['Max Temp']> 70) & (new_df['Max Temp'] <= 80)
wind_speed = new_df["Wind Speed"] < 10
cloudiness = new_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Humidity,Wind Speed,Cloudiness
31,rikitea,75.22,70,7.52,0
41,mount isa,71.60,24,9.17,0
115,kisangani,71.69,87,1.97,0
167,veinticinco de mayo,75.00,32,4.00,0
203,ancud,72.00,56,8.05,0
238,ninh binh,70.61,92,2.91,0
248,bilma,75.02,20,8.37,0
259,qeshm,71.60,73,4.70,0
297,bonthe,79.97,79,5.93,0
321,salalah,77.00,31,3.36,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = citi_df[['City','Lat','Lng','Country']]

In [23]:
hotel_df['Hotel Name']=''

<ipython-input-23-6654a56bf4d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''


In [24]:
hotel_df

,City,Lat,Lng,Country,Hotel Name
0,esperance,-33.87,121.90,AU,
1,praia da vitoria,38.73,-27.07,PT,
2,vestmannaeyjar,63.44,-20.27,IS,
3,hobart,-42.88,147.33,AU,
4,valleyview,55.07,-117.29,CA,
...,...,...,...,...,...
603,grenville,12.12,-61.62,GD,
604,bloomington,44.84,-93.30,US,
605,midyat,37.42,41.34,TR,
606,bograd,54.23,90.83,RU,


In [9]:
hotel_names = []

# set up a parameters dictionary
params = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_names.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_names.append(np.nan)

In [10]:
hotel_df['Hotel Name'] = hotel_names
hotel_df=hotel_df.dropna()
hotel_df

<ipython-input-10-d987a0a8c4bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_names


,City,Lat,Lng,Country,Hotel Name
0,esperance,-33.87,121.90,AU,Esperance Chalet Village
1,praia da vitoria,38.73,-27.07,PT,Atlântida Mar Hotel
2,vestmannaeyjar,63.44,-20.27,IS,Hotel Vestmannaeyjar
3,hobart,-42.88,147.33,AU,Hotel Grand Chancellor Hobart
4,valleyview,55.07,-117.29,CA,Paradise Inn & Suites
...,...,...,...,...,...
480,porbandar,21.64,69.61,IN,Lords Inn Porbandar
500,honiara,-9.43,159.95,SB,Coral Sea Resort & Casino
505,borovoy,55.43,86.08,RU,Tsimus
529,poronaysk,49.22,143.12,RU,Avtovokzal Khostel Stolovaya


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
fig=gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>